함수 메모장
파일 불러오기 : pd.read_csv('./tmdb_5000_credits.csv')
상위 n개 행 불러오기 : df2.head(n)
평균 : df2['행이름'].mean()
상위 100n% 값들만 필터링하기 : df2['행이름'].quantile(1 - n)
특정 행 검색 : df2.loc[조건식] (조건식 예시 : df2['행이름'] >= m)
특정 열 검색 : df2.loc['행이름']
각 행에 함수 적용 : q_movies.apply(함수명) (함수명 : 첫번째 parameter가 q_movies이여야함?)
행 정렬 : q_movies.sort_values(열name, ascending=False)



In [1]:
import pandas as pd 
import numpy as np 
df1=pd.read_csv('./tmdb_5000_credits.csv')
df2=pd.read_csv('./tmdb_5000_movies.csv')
df2.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800



데이터 셋은 https://www.kaggle.com/rounakbanik/the-movies-dataset 에서 제공하는 movies dataset을 사용했다. 
이러한 dataset을 사용한 이유로는 올인원에듀테크의 영상들과 동일한 점이 많다는 것이다. 장르에 따라 구별이 가능하고, 평점 등을 통해서 반응을 알아낼 수가 있으며, 영상이라는 공통점들이 있다. 
이 dataset을 통해서 알고리즘을 생성한 후, 올인원에듀테크의 dataset에 적용할 것이다.
movie dataset에서의 genre 종류 20가지 : 
['Action', 'Adventure', 'Fantasy', 'Science Fiction', 'Crime', 'Drama', 'Thriller', 'Animation', 'Family', 'Western', 'Comedy', 'Romance', 'Horror', 'Mystery', 'History', 'War', 'Music', 'Documentary', 'Foreign', 'TV Movie']

1. 인구통계 추천(Demographic Filtering)
 
 가장 간단한 방법이다. 유저들이 매긴 평점을 이용하여 정렬한 다음, 순위에 드는 작품들을 추천하는 방식이다. 다만 유의할 점이 있는데, 평점을 기준으로 정렬하는 것이 아니라, 한번의 계산 과정을 통해서 나온 값으로 정렬해야한다. 이러한 과정을 거치는 이유는 유저들로 인해서 평점이 계산되지만, 각 유저들의 투표는 영향이 다르기 때문이다. 
 극단적인 예시를 들어보면, 평점이 1개지만 10점인 영화와 평점이 만개지만 7점인 영화를 비교해보면 된다.
 imdb(영화 정보 제공 사이트)에서는 다음과 같은 공식을 사용한다.

 fixed_score = v * R / (v + m) + m * C / (v + m)
 v = 해당 영화의 투표 수
 m = 조건을 만족하는 최소의 투표 수
 R = 해당 영화의 평점
 C = 전체 영화의 평점 평균

 아래부터 다음 메모까지는 계산 과정이다

In [2]:
df1.columns = ['id','tittle','cast','crew']
df2 = df2.merge(df1,on='id')
df2.head(1)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,tittle,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [3]:
C= df2['vote_average'].mean()
#df2['vote_average']
C
# 영화의 전체 평균 평점은 6.09 

6.092171559442016

In [4]:
m= df2['vote_count'].quantile(0.9)
m
# vote_count 중에서 상위 10퍼에 있는 영화가 받은 투표 수

1838.4000000000015

In [5]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape
# 상위 10퍼의 data_set

(481, 23)

In [6]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [7]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)
#vote_count 상위 10% movies에 대하여, 위 함수를 적용하여 각각의 점수를 측정함

In [8]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#score 순으로 정렬함. 쇼생크 탈출, 파이트 클럽, 다크나이트 순으로 나옴
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
1881,The Shawshank Redemption,8205,8.5,8.059258
662,Fight Club,9413,8.3,7.939256
65,The Dark Knight,12002,8.2,7.920020
3232,Pulp Fiction,8428,8.3,7.904645
96,Inception,13752,8.1,7.863239
3337,The Godfather,5893,8.4,7.851236
95,Interstellar,10867,8.1,7.809479
809,Forrest Gump,7927,8.2,7.803188
329,The Lord of the Rings: The Return of the King,8064,8.1,7.727243
1990,The Empire Strikes Back,5879,8.2,7.697884
